In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import glob
import matplotlib.pyplot as plt
import sklearn
from collections import Counter
import glob
import pickle
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from mpl_toolkits.axes_grid1 import AxesGrid
from sklearn.metrics import confusion_matrix
from keras.optimizers import SGD, Adam
from random import shuffle
%matplotlib inline

In [20]:
for dir_path in glob.glob('../Bootcamp/spectrogram_final/*.png'):
    if(dir_path.split('/')[3] == '694.png'):
        continue
    img = cv2.imread(dir_path)
    #crop full audio spectograms to obtain 0-30s spectograms and 30-60s spectograms
    crop1 = img[:,0:200] #0-30 s
    crop2 = img[:,200:400] #30-60 s
    cv2.imwrite('../Bootcamp/0-30/'+dir_path.split('/')[3],crop1)
    cv2.imwrite('../Bootcamp/30-60/'+dir_path.split('/')[3],crop2)

In [3]:
map_genre = {
    0:'Pop',
    1:'Rock',
    2:'Jazz'
}

reverse_genre = {
    'Pop':0,
    'Rock':1,
    'Jazz':2
}

batch_size = 100
epochs = 5
num_classes = len(map_genre)
input_shape = (200,362,3)


def create_model_six_conv(input_shape):
    """
    CNN Keras model with 6 convolutions.
    :param input_shape: input shape, generally X_train.shape[1:]
    :return: Keras model, RMS prop optimizer
    """
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3), padding='same')) 
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    return model, opt

def load_model_from_checkpoint(weights_path, input_shape=(200,362,3)):
    model, opt = create_model_six_conv(input_shape)
    model.load_weights(weights_path)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    return model
  
def load_test_set(path):
    pics, labels = [], []
    reverse_dict = {v:k for k,v in map_characters.items()}
    for pic in glob.glob(path+'*.*'):
        char_name = "_".join(pic.split('/')[3].split('_')[:-1])
#         temp = pic.split('/')[2].split('\\')[1]
#         char_name = "_".join(temp.split('_')[:-1])
        if char_name in reverse_dict:
            temp = cv2.imread(pic)
            temp = cv2.resize(temp, (200,362)).astype('float32') / 255.
            pics.append(temp)
            labels.append(reverse_dict[char_name])
    X_test = np.array(pics)
    y_test = np.array(labels)
    y_test = keras.utils.to_categorical(y_test, num_classes) #pic in hpoen
    print("Test set", X_test.shape, y_test.shape)
    return X_test, y_test


In [4]:
X0_train, X0_test, y0_train, y0_test = [],[],[],[]
data0_data, data0_target = [],[]
data30_data, data30_target = [],[]

annotation = pd.read_csv('annotation.csv')
label = annotation['genre 1']
label = label.replace(reverse_genre)
pic_number = annotation['file_audio']
annotation['index_code'] = label

In [139]:
#len(glob.glob('../Bootcamp/0-30/*.png')) 

998

In [5]:
labelling = []
for dir_path in glob.glob('../Bootcamp/0-30/*.png'):
    numpic = dir_path.split('/')[3]
    numpic = numpic.split('.')[0]
    file_index = int(numpic)
    temp_1 = annotation[annotation['file_audio']==file_index]['index_code']
    data0_target.append(temp_1.iloc[0])
    temp1 = cv2.imread(dir_path)
    temp1 = cv2.resize(temp1, (200,362)).astype('float32')
    data0_data.append(temp1)
    
for dir_path in glob.glob('../Bootcamp/30-60/*.png'):
    numpic1 = dir_path.split('/')[3]
    numpic1 = numpic.split('.')[0]
    file_index1 = int(numpic1)
    temp_2 = annotation[annotation['file_audio']==file_index1]['index_code']
    data30_target.append(temp_2.iloc[0])
    temp2 = cv2.imread(dir_path)
    temp2 = cv2.resize(temp1, (200,362)).astype('float32')
    data30_data.append(temp1)

In [96]:
labelling1 = annotation[annotation['file_audio'] == 692]['index_code']
label15 = []
label15.append(labelling1.iloc[0])
labelling1 = annotation[annotation['file_audio'] == 100]['index_code']
label15.append(labelling1.iloc[0])
label15

[2, 1]

In [153]:
#len(data0_data)

998

In [7]:
#len(data30_target)

998

In [8]:
#train test split to split data into training, testing and labelling of data
from sklearn.model_selection import train_test_split
X0_train, X0_test, y0_train, y0_test = train_test_split(data0_data,data0_target,test_size = 0.33, random_state = 42)
X30_train, X30_test, y30_train, y30_test = train_test_split(data30_data,data30_target,test_size = 0.33, random_state = 42)

/Users/harryramli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [10]:
#0-30s 
X0_train = np.array(X0_train)
X0_test = np.array(X0_test)
y0_train = keras.utils.to_categorical(y0_train,num_classes)
y0_test = keras.utils.to_categorical(y0_test,num_classes)

#30-60s
X30_train = np.array(X30_train)
X30_test = np.array(X30_test)
y30_train = keras.utils.to_categorical(y30_train,num_classes)
y30_test = keras.utils.to_categorical(y30_test,num_classes)

In [ ]:
#creating CNN model to test 0-30s data
model , opt = create_model_six_conv(input_shape=(362,200,3))
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.fit(X0_train,y0_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X0_test,y0_test))

Train on 668 samples, validate on 330 samples
Epoch 1/5
